In [1]:
import requests
import csv
import time

In [4]:
#defining api credentials and URL, and listing us states.
apiKey = "apiKey goes here"
baseURL = "https://maps.googleapis.com/maps/api/place/textsearch/json"
states = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana",
          "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota",
          "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]
#setting queryLimit and apiCallCount to avoid exceeding usage limits.
queryLimit = 500
pause = 3
apiCallCount = 0
#defining a function to get restaurant locations.
def getLocations(state):
    global apiCallCount
    results = []
    nextPageToken = None
    while True:
        if apiCallCount >= queryLimit:
            break
        #defining parameters, initiating the api call counter and collecting the results.
        params = {"query": f"Texas Roadhouse in {state}", "key": apiKey}
        if nextPageToken:
            params["pagetoken"] = nextPageToken

        try:
            response = requests.get(baseURL, params=params)
            response.raise_for_status()
            data = response.json()
            apiCallCount += 1
            results.extend(data.get("results", []))
            nextPageToken = data.get("nextPageToken", None)
            if not nextPageToken:
                break
            time.sleep(2)
        except requests.exceptions.RequestException:
            break
    return results
#defining a function to save restaurant data to a csv file.
def saveToCSV(filename, data):
    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        # Write header
        writer.writerow(["name", "address", "state", "rating", "totalUserRatings", "placeID", "latitude", "longitude"])
        # Write each business
        for item in data:
            name = item.get("name", "N/A")
            address = item.get("formatted_address", "N/A")
            state = item.get("formatted_address", "").split(",")[-2].strip() if "," in item.get("formatted_address", "") else "N/A"
            rating = item.get("rating", "N/A")
            totalUserRatings = item.get("user_ratings_total", "N/A")
            placeID = item.get("place_id", "N/A")
            latitude = item.get("geometry", {}).get("location", {}).get("lat", "N/A")
            longitude = item.get("geometry", {}).get("location", {}).get("lng", "N/A")
            writer.writerow([name, address, state, rating, totalUserRatings, placeID, latitude, longitude])
#defining a function to fetch and save data to a csv named texasRoadhouseLocations.
def fetchAndSave():
    global apiCallCount
    allResults = []
    for state in states:
        if apiCallCount >= queryLimit:
            break
        stateResults = getLocations(state)
        allResults.extend(stateResults)
    filename = "texasRoadhouseLocations.csv"
    saveToCSV(filename, allResults)
if __name__ == "__main__":
    fetchAndSave()